In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.tensorboard import SummaryWriter

In [2]:
# Hyperparams
device = 'cuda' if torch.cuda.is_available() else 'cpu'
lr = 3e-4
z_dim = 64
image_dim = 28*28*1
batch_size = 32
epochs = 50

In [3]:
class Discriminator(nn.Module):
    def __init__(self, img_dim):
        super(Discriminator, self).__init__()
        self.disc = nn.Sequential(
            nn.Linear(img_dim, 128),
            nn.LeakyReLU(0.1),
            nn.Linear(128, 1),
            nn.Sigmoid(),
        )
        
    def forward(self, x):
        return self.disc(x)
    
    
class Generator(nn.Module):
    def __init__(self, z_dim, img_dim):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            nn.Linear(z_dim, 256),
            nn.LeakyReLU(0.1),
            nn.Linear(256, img_dim),
            nn.Tanh()
        )
        
    def forward(self, x):
        return self.gen(x)

In [4]:
disc  = Discriminator(image_dim).to(device)
gen = Generator(z_dim, image_dim).to(device)

fixed_noise = torch.randn((batch_size, z_dim)).to(device)

transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5), (0.5))
])

datasets = datasets.MNIST(root='datasets/MNIST/',transform=transforms, download=True)
loader = DataLoader(datasets, batch_size,shuffle=True)

opt_disc = optim.Adam(params=disc.parameters(), lr=lr)
opt_gen = optim.Adam(params=gen.parameters(), lr=lr)

criterion = nn.BCELoss()
writer_fake = SummaryWriter(f'logs/GAN_MNIST/fake/')
writer_real = SummaryWriter(f'logs/GAN_MNIST/real/')
step = 0

In [6]:
for epoch in range(epochs):
    for batch_idx, (real, _) in enumerate(loader):
        real = real.view(-1, 784).to(device)
        batch_size = real.shape[0]
        
        # Train Discriminator: max log(D(real)) + log(1-D(G(z)))
        noise = torch.randn(batch_size, z_dim).to(device)
        fake = gen(noise)
        disc_real = disc(real).view(-1)
        lossD_real = criterion(disc_real, torch.ones_like(disc_real)) # log(D(real))
        disc_fake = disc(fake).view(-1)
        losdD_fake = criterion(disc_fake, torch.zeros_like(disc_fake)) #  log(1-D(G(z)))
        
        lossD = (losdD_fake + lossD_real) / 2
        
        disc.zero_grad()
        lossD.backward(retain_graph=True)
        opt_disc.step()
        
        # Train Generator: min log(1 - D(G(z))) <--> max log(D(G(z)))
        output = disc(fake).view(-1)
        lossG = criterion(output, torch.ones_like(output))
        
        gen.zero_grad()
        lossG.backward()
        opt_gen.step()
        
        if batch_idx == 0:
            print(
                f"Epoch [{epoch}/{epochs}] Batch {batch_idx}-{len(loader)} \
                      Loss D: {lossD:.4f}, loss G: {lossG:.4f}"
            )

            with torch.no_grad():
                fake = gen(fixed_noise).reshape(-1, 1, 28, 28)
                data = real.reshape(-1, 1, 28, 28)
                img_grid_fake = torchvision.utils.make_grid(fake, normalize=True)
                img_grid_real = torchvision.utils.make_grid(data, normalize=True)

                writer_fake.add_image(
                    "Mnist Fake Images", img_grid_fake, global_step=step
                )
                writer_real.add_image(
                    "Mnist Real Images", img_grid_real, global_step=step
                )
                step += 1

Epoch [0/50] Batch 0/1875                       Loss D: 0.2445, loss G: 1.6099
Epoch [1/50] Batch 0/1875                       Loss D: 0.2714, loss G: 1.6533
Epoch [2/50] Batch 0/1875                       Loss D: 0.6335, loss G: 0.8346
Epoch [3/50] Batch 0/1875                       Loss D: 0.6821, loss G: 0.8007
Epoch [4/50] Batch 0/1875                       Loss D: 0.5442, loss G: 1.2814
Epoch [5/50] Batch 0/1875                       Loss D: 0.6906, loss G: 0.9932
Epoch [6/50] Batch 0/1875                       Loss D: 0.6252, loss G: 0.8267
Epoch [7/50] Batch 0/1875                       Loss D: 0.4991, loss G: 1.4214
Epoch [8/50] Batch 0/1875                       Loss D: 0.5278, loss G: 0.9929
Epoch [9/50] Batch 0/1875                       Loss D: 0.7174, loss G: 1.1433
Epoch [10/50] Batch 0/1875                       Loss D: 0.4942, loss G: 1.2458
Epoch [11/50] Batch 0/1875                       Loss D: 0.5973, loss G: 1.3455
Epoch [12/50] Batch 0/1875                       L